In [1]:
# Here we demonstrate how to randomly generate clusters and adsorption sites

In [2]:
import numpy as np
from ase.db import connect
from ase.io import read, write
from ase.atoms import Atoms
from ase.visualize import view

In [3]:
# we have to pick a given cluster size
N = 50

# now we have to define the stochiometric ratios
# we'll exemplify with a 50:50 split

AgPd_base = Atoms('Ag'*25 + 'Pd'*25, cell=[20,20,20], pbc=False)

In [4]:
# now we need to define the atomic positions
# the most straight forward approach is to randomly generate them
# and then relax them, such that they form reasonable starting configurations

# we will generate N random 3D points accounting for contraints
# the basic constraints are atoms shouldn't be too close, nor too far

# define box within which to generate positions (Angstroms)
box = 10 

# define the basic minimal and maximal inter-atomic distance constraints
dmin = 2.5
dmax = np.sqrt(3)*box

In [5]:
# here we generate possible positions one by one,
# add them to the set if the constraints are satisfied, and loop

# you can run this over and over until it works

# note that the constraints must make sense
# relative to the random numbers generated!

def genCluster(test):

    cluster = test.copy()
    iterations = 10000
    positions = []
    
    while len(positions) < N and iterations>0:
    
        point = np.random.uniform(0, box, size=3)
        if positions:
            distances = np.linalg.norm(np.array(positions)-point, axis=1)
            if not all(dmin < d < dmax for d in distances):
                iterations -= 1
                continue
        positions.append(point)
        iterations-=1
    
    if iterations>0: 
        cluster.positions = positions
        cluster.center()
    else: print('failed')
        
    return cluster

In [6]:
AgPd_gen = genCluster(AgPd_base)
view(AgPd_gen, viewer='x3d')

In [7]:
# To make this cluster more reasonable, we can relax it with a simulation

In [8]:
# this requires a calculator (EMT, Lammps, NN, etc)
# and an optimizer (BFGS, FIRE, etc)

from ase.calculators.emt import EMT
from ase.optimize import BFGS

def relaxCluster(test):
    
    cluster = test.copy()
    cluster.calc = EMT()
    sim = BFGS(cluster, trajectory=None, logfile=None)
    
    if sim.run(fmax=0.1, steps=500):
        return cluster
    else: print('failed'); return test

# note that you can remove 'logfile=None'
# if you want to inspect the relaxation process

In [9]:
AgPd_relax = relaxCluster(AgPd_gen)
view(AgPd_relax, viewer='x3d')

In [10]:
# Now we showcase how to randomly generate adsorption sites

In [11]:
# first define your adsorbate, 
# either by what is available or your own
# here we exemplify the process with water

from ase.build import molecule

adsorbate = molecule('H2O')
adsorbate.cell = [20,20,20]

In [12]:
# define radial range from center of cluster
# these values are parameterized to N (atom count) and the cell size
rmin = 7
rmax = 10

# define constraint for adsorption height if desired
adsmin = 4.5
adsmax = 5.0

assert np.abs(rmin-rmax) >= np.abs(adsmin-adsmax)
# due to the amorphous surface height of our clusters,
# we should generate positions within a broader radius,
# and only thereafter filter for more specific adsorption heights

def gensite(test):
    
    # we can generate an adsorbtion site like this:
    # first randomly generate the radius relative to the center
    # then generate theta and phi, and finally convert to xyz
    # we can loop this process until the constraints are satisfied

    cluster = test.copy()
    iterations = 1000
    
    good = False
    while not good and iterations>0:

        ads = adsorbate.copy()
        ads.center()
        
        # generate adsorbate point
        r = np.cbrt(np.random.uniform(rmin**3, rmax**3))
        theta = np.random.uniform(0, np.pi)
        phi = np.random.uniform(0, 2*np.pi)
        x, y, z = np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)
        point = [r*x,r*y,r*z]

        # update adsorbate position
        ads.positions += point
        
        # rotate adsorbate randomly about its center of mass
        com = np.mean(ads.positions, axis=0)
        ads.rotate(np.random.uniform(0, 180), 'x', center=com)
        ads.rotate(np.random.uniform(0, 180), 'y', center=com)
        ads.rotate(np.random.uniform(0, 180), 'z', center=com)

        # combine the cluster and adsorbate
        both = cluster + ads
        
        # check inter-atomic constraints
        dists = np.mean([d[:-len(ads)] for d in both.get_all_distances()[N:]], axis=0)
        if all(dmin < d < dmax for d in dists):

            # we can also investigate the adsorbate's nearest neighbor(s)
            index = np.argsort(dists)
            stoi = cluster.get_chemical_symbols()

            nn = stoi[index[0]]
            height = dists[index[0]]

            # and or ensure the nn is of a specific element
            #if (adsmin < height < adsmax) and nn=='Pd':
            #    good = True

            # example for requiring multiple (3) same-element nns
            #nns = [stoi[i] for i in index[:3]]
            #if (adsmin < height < adsmax) and all(n=='Pd' for n in nns): 
            #    good = True

            # example to ensure a specific kind of site
            # note that these values need to be calibrated
            smin, smax = adsmin, adsmin+(adsmax-adsmin)*0.5
            count = np.sum((dists >= smin) & (dists <= smax))
            if count==1: site = 'top'
            elif count==2: site = 'bridge'
            elif count==3: site = 'hollow'
            else: site = 'other'
            if (adsmin < height < adsmax) and site=='hollow': 
                good = True
        
        else: iterations -=1

    if iterations==0: print('failed')
    print(iterations)
    return both

# note that we've re-used dmin and dmax inter-atomic constraints defined earlier

In [13]:
AgPd_ads = gensite(AgPd_relax)
view(AgPd_ads, viewer='x3d')

974


In [14]:
# That's it!
# perhaps we want to move a given adsorbate closer, without having to generate it?
# we could center the cluster+adsorbate's center of mass on zero, then multiply the positions
# then re-center the cluster in the box's center, or we could simply relax a couple steps

In [15]:
from ase.constraints import FixAtoms, FixBondLengths
from ase.calculators.emt import EMT
from ase.optimize import BFGS

def relaxads(test, steps):
    
    cluster = test.copy()
    cluster.calc = EMT()
    cluster_indices = np.arange(N)
    ads_count = int(len(cluster)-N)

    # we can optionally maintain the adsorbate bond lengths
    # however this requires knowing the bond order if > 1
    # e.g., for H2O, the stoichiometry is OH2, hence bond_indices = [[N,N+1], [N,N+2]] 

    bond_indices = []
    if ads_count==2: bond_indices = np.arange(N,len(cluster))
    elif ads_count>2:
        print('fixing complex bonds requires knowing bond orders relative to stoichiometry')
        # here we assume H2O
        bond_indices = [[N,N+1], [N,N+2]]

    cluster.set_constraint([FixAtoms(cluster_indices), FixBondLengths(bond_indices)])

    i_height = np.sort(np.mean([d[:-ads_count] for d in cluster.get_all_distances()[N:]], axis=0))[0]
    sim = BFGS(cluster, trajectory=None, logfile=None)
    sim.run(fmax=0.001, steps=steps)
    f_height = np.sort(np.mean([d[:-ads_count] for d in cluster.get_all_distances()[N:]], axis=0))[0]

    print('height changed from', i_height, 'to', f_height)
    cluster.set_constraint(None)
    return cluster

# note that we rely on cluster size N as defined earlier
# note that this relaxation can actually push the adsorbate away!

In [16]:
AgPd_closer = relaxads(AgPd_ads, steps=100)
view(AgPd_closer, viewer='x3d')

fixing complex bonds requires knowing bond orders relative to stoichiometry
height changed from 4.519010588579781 to 2.889065800283125


In [17]:
###